## 分布式ID

### 生成分布式ID的9种方法

* UUID
* 数据库自增ID
* 数据库多主模式
* 号段模式
* Redis
* 雪花算法（SnowFlake）
* 滴滴出品（TinyID）
* 百度 （Uidgenerator）
* 美团（Leaf）

**1. UUID**
* 将数据库中的ID试用UUID生成
* 生成足够简单，本地生成无网络消耗，具有唯一性
* **缺点**：
    * 无序的字符串，不具备趋势自增特性
    * 没有具体的业务含义
    * 长度过长16 字节128位，36位长度的字符串，存储以及查询对MySQL的性能消耗较大，MySQL官方明确建议主键要尽量越短越好，作为数据库主键 UUID 的无序性会导致数据位置频繁变动，严重影响性能。

**2. 基于数据库自增ID**
* 用一个单独的MySQL实例（数据库中的表）用来生成ID， 当我们需要一个ID时，就往这个表中插入一个数据获取ID；
* 实现简单，ID单调自增，数值类型查询速度快
* **缺点：**DB单点存在宕机风险，无法扛住高并发场景

**3. 基于数据库集群模式**
* 使用集群的方式，主从模式生产ID
* 需要设置每个数据库的【起点】，【步长】
* 解决了单点故障的问题；
* 但是不利于扩展，也无法满足高并发的场景

**4. 基于数据库的号段模式**
* 类似于2，但是不同的是，每次去获取的时候，获取一批ID
* 然后数据库先使用申请到的ID，当不够用时，再去申请新的；
* 通过【段长度】，【起始地址】，【版本】控制ID生成也并发，版本号是以CAS这种乐观锁的方式进行修改的

**5. 基于Redis模式**
* 将2中的数据库变成Redis而已，需要考虑Redis的持久化
* 如果Redis宕机，也可能出现重复ID的情况

**6. 基于雪花算法（Snowflake）模式**

* 生成的是Long类型的ID，一个Long类型占8个字节，每个字节占8比特，也就是说一个Long类型占64个比特。


Snowflake ID组成结构：正数位（占1比特）+ 时间戳（占41比特）+ 机器ID（占5比特）+ 数据中心（占5比特）+ 自增值（占12比特），总共64比特组成的一个Long类型。

* 第一个bit位（1bit）：Java中long的最高位是符号位代表正负，正数是0，负数是1，一般生成ID都为正数，所以默认为0。
* 时间戳部分（41bit）：毫秒级的时间，不建议存当前时间戳，而是用（当前时间戳 - 固定开始时间戳）的差值，可以使产生的ID从更小的值开始；41位的时间戳可以使用69年，(1L << 41) / (1000L * 60 * 60 * 24 * 365) = 69年
* 工作机器id（10bit）：也被叫做workId，这个可以灵活配置，机房或者机器号组合都可以。
* 序列号部分（12bit），自增值支持同一毫秒内同一个节点可以生成4096个ID
